In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.1 MB/s eta 0:00:00


#TIFF to PNG

In [ ]:
import rasterio
from rasterio.plot import reshape_as_image
import numpy as np
import imageio

def convert_geotiff_to_png(input_path, output_path):
    with rasterio.open(input_path) as dataset:
        # Read the image bands
        data = dataset.read()

        # Normalize the image data to the range of 0 to 1
        normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))

        # Reshape the normalized image bands to a 2D array
        image = reshape_as_image(normalized_data)

        # Save the image as PNG without alpha channel
        imageio.imwrite(output_path, (image * 255).astype(np.uint8))

# Usage example
'''
input_path = '/content/drive/MyDrive/earthengine/AobjectShadowsMasked.tif'
output_path = '/content/drive/MyDrive/Temporary/AobjectShadowsMasked.png'
convert_geotiff_to_png(input_path, output_path)

input_path = '/content/drive/MyDrive/earthengine/KObjectShadowsMasked.tif'
output_path = '/content/drive/MyDrive/Temporary/KObjectShadowsMasked.png'
convert_geotiff_to_png(input_path, output_path)
'''

#Splitting


In [ ]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from PIL import Image

def split_image(image_path, output_directory, piece_size):
    # Open the image using PIL
    image = Image.open(image_path)

    # Get the dimensions of the image
    width, height = image.size

    # Calculate the number of pieces in both dimensions
    num_pieces_x = width // piece_size
    num_pieces_y = height // piece_size
    n = 0
    # Iterate over the image and extract each piece
    for y in range(num_pieces_y):
        for x in range(num_pieces_x):
            # Calculate the coordinates for each piece
            left = x * piece_size
            upper = y * piece_size
            right = left + piece_size
            lower = upper + piece_size

            # Crop the image to extract the piece
            piece = image.crop((left, upper, right, lower))

            # Save the piece with a unique name
            output_path = f"{output_directory}/piece_{n}.png"
            piece.save(output_path)
            n = n + 1

# Usage examples
'''
input_image_path = '/content/drive/MyDrive/Temporary/ACoperBeforeSplitResized.png'
output_directory = '/content/drive/MyDrive/Temporary/Coper'
piece_size = 1024

split_image(input_image_path, output_directory, piece_size)

input_image_path = '/content/drive/MyDrive/Samhällsbyggnadsprogrammet/KEX/Results/KObjectBasedResized.png'
output_directory = '/content/drive/MyDrive/Temporary/Object'

split_image(input_image_path, output_directory, piece_size)
'''

All images have 7 images on each row, meaning that the first row starts on image 0 and the second one starts on image 7


#Re-merge images

Antioch has 70 images, meaning x = 7, y = 10

In [ ]:
from PIL import Image
import os

def extract_number_from_filename(filename):
    # Extract the number from the filename
    return int(os.path.splitext(filename)[0].split("_")[1])

def combine_images(input_directory, output_image_path, images_per_row):
    # Get a list of all the image files in the input directory
    image_files = sorted(os.listdir(input_directory), key=extract_number_from_filename)

    # Open the first image to get the dimensions
    first_image = Image.open(os.path.join(input_directory, image_files[0]))
    width, height = first_image.size

    # Calculate the number of rows and columns
    num_images = len(image_files)
    num_rows = (num_images + images_per_row - 1) // images_per_row
    num_columns = min(num_images, images_per_row)

    # Create a new blank image to hold the combined pieces
    combined_image = Image.new("RGBA", (width * num_columns, height * num_rows))

    # Iterate over the image files and paste each piece onto the combined image
    for i, image_file in enumerate(image_files):
        piece = Image.open(os.path.join(input_directory, image_file))
        x = i % images_per_row
        y = i // images_per_row
        left = x * width
        upper = y * height
        combined_image.paste(piece, (left, upper))

    # Save the combined image
    combined_image.save(output_image_path)

# Usage examples
'''
input_directory = "/content/drive/MyDrive/Temporary/Coper"
output_image_path = "/content/drive/MyDrive/Temporary/ACoperRemergedV2.png"
images_per_row = 7
combine_images(input_directory, output_image_path, images_per_row)

input_directory = '/content/drive/MyDrive/Temporary/Object'
output_image_path = "/content/drive/MyDrive/Temporary/ObjectRemerged.png"
combine_images(input_directory, output_image_path, images_per_row)
'''